This Pipeline is based on the computer vision lab where the functions were modified for the purpose of this lab

In [1]:
import numpy as np
import os
import pandas as pd
from scipy.stats import zscore
from sklearn.metrics import r2_score
#from sklearn.model_selection import train_test_split
from numpy.random import RandomState

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
SEED= 123456789
BATCH_SIZE = 10
NUM_WORKERS = 4
NUM_EPOCHS = 100
VALIDATION_INDEX = 750 #We will take the VALIDATION_INDEX firts samples as training and the remaining as validation

In [3]:
# Updating mean class for loss aggregation.
class UpdatingMean():
    def __init__(self):
        self.sum = 0
        self.n = 0

    def mean(self):
        return self.sum / self.n

    def add(self, loss):
        self.sum += loss
        self.n += 1

In [4]:
class MRIDataset(Dataset):
    def __init__(self, train=True):
        super().__init__()
        
        X_train_test = pd.read_csv('data/X_train_cleaned.csv')
        y_train_test = pd.read_csv('data/y_train_cleaned.csv')
        
        #Split in test and validation:
        prng = RandomState(SEED)
        train_mask = prng.rand(len(X_train_test)) < 0.8 #~80% of training and 20% of validation
                
        if(train):
            self.samples = X_train_test[train_mask].to_numpy()
            self.annotations= y_train_test[train_mask].to_numpy()
        else:
            self.samples = X_train_test[~train_mask].to_numpy()
            self.annotations= y_train_test[~train_mask].to_numpy()
                                        
    def __len__(self):
        # Returns the number of samples in the dataset.
        return self.samples.shape[0]
    
    def __getitem__(self, idx):
        # Returns the sample and annotation with index idx.
        sample = self.samples[idx]
        annotation = self.annotations[idx]
                
        # Convert to tensor.
        return {
            'input': torch.from_numpy(sample).float(),
            'annotation': torch.from_numpy(annotation).float()
        }

In [5]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else torch.zeros(1) #What to do when divided by zero?
    return r2

In [6]:
def run_training_epoch(net, optimizer, dataloader):
    loss_aggregator = UpdatingMean()
    # Put the network in training mode.
    net.train()
    # Loop over batches.
    for batch in dataloader:
        # Reset gradients.
        optimizer.zero_grad()

        # Forward pass.
        X = batch["input"]
        y = batch["annotation"]
        output = net(X)

        # Compute the loss - MAE
        loss = nn.MSELoss()
        output_loss = loss(output, y)

        # Backwards pass.
        output_loss.backward()
        optimizer.step()

        # Save loss value in the aggregator.
        loss_aggregator.add(output_loss)
    return loss_aggregator.mean()


def compute_accuracy(output, labels):
    return r2_loss(output, labels)


def run_validation_epoch(net, dataloader):
    accuracy_aggregator = UpdatingMean()
    # Put the network in evaluation mode.
    net.eval()
    # Loop over batches.
    for i, batch in enumerate(dataloader):
        # Forward pass only.
        output = net(batch['input'])

        # Compute the accuracy using compute_accuracy.
        accuracy = compute_accuracy(output.detach().numpy(), batch['annotation'])

        # Save accuracy value in the aggregator.
        accuracy_aggregator.add(accuracy.item())
    return accuracy_aggregator.mean()

In [47]:
class MLPTest(nn.Module):
    def __init__(self):
        super().__init__()

        self.codename = 'mlp'

        # Define the network layers in order.
        # Input is 199D.
        # Output is a single value.
        # Multiple linear layers each followed by a LeakyReLU non-linearity.
        self.layers = nn.Sequential(
            nn.Linear(199, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 10),
            nn.LeakyReLU(),
            nn.Linear(10, 1),
            #nn.LeakyReLU(),
            #nn.Linear(5, 1),
            nn.LeakyReLU()
        )
    
    def forward(self, batch):
        # Process batch using the layers.
        x = self.layers(batch)
        return x

In [48]:
# Create the training dataset and dataloader.
train_dataset = MRIDataset(train=True)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True
)
# Create the validation dataset and dataloader.
valid_dataset = MRIDataset(train=False)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

# Create the network.
net = MLPTest()

# Create the optimizer., lr=0.005
optimizer = Adam(net.parameters())

# Main training loop.
best_accuracy = 0
for epoch_idx in range(NUM_EPOCHS):
    # Training code.
    loss = run_training_epoch(net, optimizer, train_dataloader)
    print('[Epoch %02d] Loss: %.4f' % (epoch_idx + 1, loss))

    # Validation code.
    acc = run_validation_epoch(net, valid_dataloader)
    print('[Epoch %02d] r2_score.: %.4f' % (epoch_idx + 1, acc))

    # Save checkpoint if accuracy is the best so far.
    checkpoint = {
        'epoch_idx': epoch_idx,
        'net': net.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    if acc > best_accuracy:
        best_accuracy = acc
        torch.save(checkpoint, f'best-{net.codename}.pth')


[Epoch 01] Loss: 4771.1499
[Epoch 01] r2_score.: -68.4152
[Epoch 02] Loss: 3982.5259
[Epoch 02] r2_score.: -47.7271
[Epoch 03] Loss: 2578.4617
[Epoch 03] r2_score.: -29.3501
[Epoch 04] Loss: 2031.9648
[Epoch 04] r2_score.: -26.1725
[Epoch 05] Loss: 1865.7903
[Epoch 05] r2_score.: -25.1557
[Epoch 06] Loss: 1782.5873
[Epoch 06] r2_score.: -24.4645
[Epoch 07] Loss: 1747.7057
[Epoch 07] r2_score.: -23.8998
[Epoch 08] Loss: 1755.4540
[Epoch 08] r2_score.: -23.3756
[Epoch 09] Loss: 1648.3805
[Epoch 09] r2_score.: -22.8410
[Epoch 10] Loss: 1560.6420
[Epoch 10] r2_score.: -22.1052
[Epoch 11] Loss: 1496.1311
[Epoch 11] r2_score.: -21.5625
[Epoch 12] Loss: 1438.6906
[Epoch 12] r2_score.: -20.8191
[Epoch 13] Loss: 1392.0734
[Epoch 13] r2_score.: -19.8008
[Epoch 14] Loss: 1303.6826
[Epoch 14] r2_score.: -18.2815
[Epoch 15] Loss: 1210.5154
[Epoch 15] r2_score.: -16.7962
[Epoch 16] Loss: 1092.9053
[Epoch 16] r2_score.: -15.0745
[Epoch 17] Loss: 992.2244
[Epoch 17] r2_score.: -13.6112
[Epoch 18] Loss

In [50]:
X_train_test = pd.read_csv('data/X_train_cleaned.csv')
y_train_test = pd.read_csv('data/y_train_cleaned.csv')

prng = RandomState(SEED)
train_mask = prng.rand(len(X_train_test)) < 0.8 #~80% of training and 20% of validation
X_validation = X_train_test[~train_mask].to_numpy()
y_validation = y_train_test[~train_mask].to_numpy()

#output = net(torch.from_numpy(X_validation).float())
output = torch.ceil(net(torch.from_numpy(X_validation).float())) #Gives better result cause our NN slightly underestimate our result
r2_score(output.detach().numpy(),  y_validation)

0.35426631645776574